# STATS401 Project 2


## Data Acquisition
---

### Query the transaction data of CryptoPunk from [Dune Analytics](https://dune.com/browse/queries/authored)


In [10]:
!pip install duneanalytics


In [11]:
from duneanalytics import DuneAnalytics
import time
import json
import pandas as pd
import numpy as np


In [12]:
def query_data_from_dune(data_mode):
    """
    Query data from Dune Analytics
    """
    # initialize client
    dune = DuneAnalytics('brucezzzzzzzyf@gmail.com', 'ZHANG0509yf')

    # try to login
    dune.login()

    # fetch token
    dune.fetch_auth_token()

    # set query ids
    if data_mode == 'punkBought':
        query_dict = {'2017': 673785,
                      '2018': 673784,
                      '2019': 673783,
                      '2020': 673781,
                      '2021': 673779,
                      '2022': 673771}
    elif data_mode == 'punkTransfer':
        query_dict = {'2017-2019': 674456,
                      '2020': 674472,
                      '2021': 674496,
                      '2022': 674500}

    # query data
    dune_data = {}
    for year in query_dict:
        query_id = query_dict[year]
        result_id = dune.query_result_id(query_id=query_id)
        dune_data[year] = dune.query_result(result_id)
        time.sleep(5)

    return dune_data


def save_data(dune_dict, file_predix, DUNE_DATA_PATH='./dune_data'):
    """
    Write data to json file
    """
    from datetime import date
    today = str(date.today())
    out_file = '{}/{}_{}.json'.format(DUNE_DATA_PATH, file_predix, today)
    with open(out_file, 'w') as f:
        json.dump(dune_dict, f)
    print('Data saved to {}'.format(out_file))


In [ ]:
dune_bt_dict = query_data_from_dune('punkBought')
save_data(dune_bt_dict, 'dune_bt_dict')


In [ ]:
dune_tf_dict = query_data_from_dune('punkTransfer')
save_data(dune_tf_dict, 'dune_tf_dict')


## Data Cleaning
---

In [1]:
import time
import json
import pandas as pd
import numpy as np

In [2]:
dune_bt_dict = json.load(open('./dune_data/dune_bt_dict_2022-05-06.json'))
dune_tf_dict = json.load(open('./dune_data/dune_tf_dict_2022-05-06.json'))

In [3]:
def index_trader(buyer, seller):
    trader = list(set(list(buyer.unique()) + list(seller.unique())))
    # print('Total {} traders'.format(len(trader)))

    return {trader[i]: i for i in range(len(trader))}


### 1. Transaction Database

- **tx_id**
- date_time
- from: trader_id
- yo: trader_id
- eth_price
- punk_id: punk_id


In [4]:
def create_tx_db(dune_bt_dict, dune_tf_dict):
    temp_idx = 0
    # punkBought data frame
    bt_data = dict()
    for year in dune_bt_dict:
        data_list = dune_bt_dict[year]['data']['get_result_by_result_id']
        bt_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # punkTransfer data frame
    tf_data = dict()
    for year in dune_tf_dict:
        data_list = dune_tf_dict[year]['data']['get_result_by_result_id']
        tf_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # concatenate data frames
    tx_db = pd.concat([pd.DataFrame.from_dict(bt_data, orient='index'),
                       pd.DataFrame.from_dict(tf_data, orient='index')])

    # set date_time
    tx_db['date_time'] = pd.to_datetime(tx_db['_date'] + ' ' + tx_db['_time'])

    # remove txs whose buyer is \x0000000000000000000000000000000000000000
    tx_db = tx_db[tx_db['buyer'] !=
                  '\\x0000000000000000000000000000000000000000']

    # reindex seller and buyer using trader_id
    trader_index_dict = index_trader(tx_db['buyer'], tx_db['seller'])
    tx_db['from'] = tx_db['seller'].apply(lambda x: trader_index_dict[x])
    tx_db['to'] = tx_db['buyer'].apply(lambda x: trader_index_dict[x])

    # sort by date_time
    tx_db.sort_values(by='date_time', inplace=True)

    # index transaction
    tx_db['tx_id'] = range(len(tx_db))
    tx_db = tx_db.loc[:, ['tx_id', 'date_time',
                          'from', 'to', 'eth_price', 'punk_id']]
    tx_db.set_index('tx_id', inplace=True)

    print('Total {} transactions'.format(len(tx_db)))
    print('Total {} unique traders'.format(len(trader_index_dict)))
    print('Total {} unique punk_id'.format(len(tx_db['punk_id'].unique())))

    return tx_db, trader_index_dict


tx_db, trader_index_dict = create_tx_db(dune_bt_dict, dune_tf_dict)
tx_db


Total 40540 transactions
Total 8907 unique traders
Total 9007 unique punk_id


,date_time,from,to,eth_price,punk_id
tx_id,,,,,
0,2017-06-23 21:06:32,5693,3115,0.01,3134
1,2017-06-23 21:06:45,5693,5842,0.04,5719
2,2017-06-23 21:06:53,3115,1049,0.10,5056
3,2017-06-23 22:06:04,5693,8666,NaN,8138
4,2017-06-23 22:06:08,5693,5842,0.06,5624
...,...,...,...,...,...
40535,2022-05-05 21:05:57,4355,7658,70.00,3314
40536,2022-05-06 02:05:19,7562,6983,NaN,3904
40537,2022-05-06 03:05:13,7739,3502,NaN,4332


### 2. CryptoPunk

- **punk_id**
- img_url
- type (Alien, Ape, Zombie, Female, Male)
- skin_tone
- attr_count
- attributes
- current_owner: trader_id
- avg_price


In [57]:

def create_punk_db(punk_info_csv_list, CSV_PATH='./cp', PUNK_IMG_PATH='./punk_imgs'):
    punk_db = pd.read_csv('{}/{}'.format(CSV_PATH, punk_info_csv_list[0]))
    for i in range(1, len(punk_info_csv_list)):
        punk_df_temp = pd.read_csv(
            '{}/{}'.format(CSV_PATH, punk_info_csv_list[i]))
        punk_db = pd.concat([punk_db, punk_df_temp], axis=0)

    # rename columns
    punk_db.columns = ['punk_id', 'type', 'gender',
                       'skin_tone', 'attr_count', 'attributes']

    # strip type, gender, skin_tone
    punk_db['type'] = punk_db['type'].apply(lambda x: x.strip())
    punk_db['gender'] = punk_db['gender'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(lambda x: x.strip())
    
    # set cryptopunk image dir
    punk_db['img_dir'] = punk_db['punk_id'].apply(
        lambda x: '{}/{}.png'.format(PUNK_IMG_PATH, x))
    
    # make attributes as list
    punk_db['attributes'] = punk_db['attributes'].apply(lambda x: [i.strip() for i in x.split('/')])

    # set punk_id as index
    punk_db.set_index('punk_id', inplace=True)

    return punk_db


punk_info_csv_list = ['0-999.csv', '1000-1999.csv', '2000-2999.csv', '3000-3999.csv',
                      '4000-4999.csv', '5000-5999.csv', '6000-6999.csv',
                      '7000-7999.csv', '8000-8999.csv', '9000-9999.csv']

punk_db = create_punk_db(punk_info_csv_list)
punk_db


,type,gender,skin_tone,attr_count,attributes,img_dir
punk_id,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",./punk_imgs/0.png
1,Human,Male,Dark,2,"[Smile, Mohawk]",./punk_imgs/1.png
2,Human,Female,Light,1,[Wild Hair],./punk_imgs/2.png
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",./punk_imgs/3.png
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",./punk_imgs/4.png
...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",./punk_imgs/9995.png
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",./punk_imgs/9996.png
9997,Zombie,Male,,2,"[Front Beard, Cap Forward]",./punk_imgs/9997.png


### 3. Trader

- **trader_id**
- address
- tx_involved_count
- tx_involved: list of tx_id
- frequent_skin_tone
- frequent_gender


In [6]:

def find_most_frequent_skin_tone(tx_involved_list, tx_db):
    skin_tones = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        skin_tones.append(punk_db.loc[punk_id, 'skin_tone'])
    return max(set(skin_tones), key=skin_tones.count)


def find_most_frequent_gender(tx_involved_list, tx_db):
    genders = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        genders.append(punk_db.loc[punk_id, 'gender'])
    return max(set(genders), key=genders.count)


def create_trader_db(tx_db, trader_index_dict):
    trader_db = pd.DataFrame.from_dict(
        trader_index_dict, orient='index', columns=['trader_id'])

    # set address for each trader
    trader_db.reset_index(inplace=True)
    trader_db.rename(columns={'index': 'address'}, inplace=True)

    # set trader_id as index
    trader_db.sort_values(by='trader_id', inplace=True)

    # get the involved tx
    trader_db['tx_involved_count'] = trader_db['trader_id'].apply(
        lambda x: len(tx_db[tx_db['from'] == x]) + len(tx_db[tx_db['to'] == x]))
    trader_db['tx_involved'] = trader_db['trader_id'].apply(lambda x: sorted(list(tx_db[tx_db['from'] == x].index) +
                                                            list(tx_db[tx_db['to'] == x].index)))

    # get the skin tone that each trader most frequently trades with
    trader_db['frequent_skin_tone'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_skin_tone(x, tx_db))

    # get the gender that each trader most frequently trades with
    trader_db['frequent_gender'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_gender(x, tx_db))

    # set trader_id as index
    trader_db.set_index('trader_id', inplace=True)

    return trader_db


trader_db = create_trader_db(tx_db, trader_index_dict)
trader_db


,address,tx_involved_count,tx_involved,frequent_skin_tone,frequent_gender
trader_id,,,,,
0,\x7cd3fc0e4ef95897f9b8db951dfc075cc924ebbe,1,[29443],,Male
1,\x020cf21cb58942e2c2fa355c4646d61b576e19c0,2,"[26662, 38004]",Dark,Male
2,\x3d8680de4fbf1bc1bf2711369826e0385e88f675,12,"[28215, 28774, 30003, 30273, 30343, 30840, 308...",Medium,Male
3,\x603b93e94b997598d642b3fcf28fb4841c24634a,2,"[22542, 23109]",Light,Male
4,\x6130d3edf972ef4bd1fa7ebaa2458b34e5edac49,38,"[12143, 12164, 15130, 15693, 15694, 15695, 156...",Light,Male
...,...,...,...,...,...
8902,\x94eba34e650787b6dbf89f9dd0d31923521dc9e3,1,[39161],Dark,Female
8903,\x1b7fcc22d66f1e2573da904adc457fd104ab72b3,1,[33629],Light,Female
8904,\x560b9244a2fdc11563a7e7293997d54e147d63c3,5,"[552, 562, 563, 564, 565]",Light,Female


## Data Engineering for Visualization
---

In [77]:
VIS_DATA_PATH = './vis_data'

### 1. Sankey Diagram

<img src="imgs/vis1_example.png" alt="drawing" width="500">

Data format

```
{
    "nodes":[
        {"node":0,"name":"Human"},
        {"node":1,"name":"Zombie"},
        {"node":2,"name":"Ape"},
        {"node":3,"name":"Alien"},

        {"node":4,"name":"Female"},
        {"node":5,"name":"Male"},

        {"node":6,"name":"Medium"},
        {"node":7,"name":"Dark"},
        {"node":8,"name":"Light"},
        {"node":9,"name":"Albino"},
        {"node":10,"name":"non-human"},

        {"node":11,"name":"Beanie"},
        ...
        {"node":97,"name":"Stringy Hair"}
    ],

    "links":[
        {"source":<node_id>, "target":<node_id>, "value":<count>},
        ...
    ]
}
```

In [83]:
def between_skin_tone_and_attributes(item, source_skin_tone, target_attribute):
    return (item['skin_tone'] == source_skin_tone) and (target_attribute in item['attributes'])


def get_dataset_vis1(tx_db, punk_db, trader_db):
    dataset_vis1 = dict()

    # nodes
    node_names_nested_list = list()
    # get the list of all types
    # ['Human', 'Zombie', 'Ape', 'Alien']
    type_list = list(punk_db['type'].unique())
    node_names_nested_list.append(type_list)

    # get the list of all genders
    gender_list = list(punk_db['gender'].unique()
                       )          # ['Female', 'Male']
    node_names_nested_list.append(gender_list)

    # get the list of all skin_stones
    # ['Medium', 'Dark', 'Light', 'Albino', '']
    skin_tone_list = list(punk_db['skin_tone'].unique())
    skin_tone_list.remove('')
    # skin_tone_list = [
    #     x if x != '' else 'Unknown_skin_tone' for x in skin_tone_list]
    node_names_nested_list.append(skin_tone_list)

    # get the list of all attributes
    attribute_list = list(punk_db.attributes)
    attribute_list = list(
        set([item for elem in attribute_list for item in elem]))
    attribute_list = sorted(attribute_list)
    attribute_list.remove('')
    # attribute_list = [
    #     x if x != '' else 'No attributes' for x in attribute_list]
    node_names_nested_list.append(attribute_list)

    # set node_id and node_name
    node_list = list()

    node_names = type_list + gender_list + attribute_list
    for name in node_names:
        node_list.append({'node': len(node_list), 'name': name})

    dataset_vis1['nodes'] = node_list
    print('> Saved: nodes')

    # links
    link_list = list()
    levels_list = ['type', 'gender', 'skin_tone']
    # links between each pair of neighbor levels
    for i in range(len(levels_list)-1):
        source_level = levels_list[i]
        source_name_list = node_names_nested_list[i]

        target_level = levels_list[i+1]
        target_name_list = node_names_nested_list[i+1]

        for source in source_name_list:
            for target in target_name_list:
                value = punk_db[(punk_db[source_level] == source) & (
                    punk_db[target_level] == target)].shape[0]
                link_list.append(
                    {'source': source, 'target': target, 'value': value})
    print('>> Saved: links between type, gender, and skin_tone')

    # links between skin_tone and attributes
    for source in skin_tone_list:
        for target in attribute_list:
            value = punk_db[punk_db.apply(lambda x: between_skin_tone_and_attributes(
                x, source, target), axis=1)].shape[0]
            link_list.append(
                {'source': source, 'target': target, 'value': value})
    print('>> Saved: links between skin_tone and attributes')

    dataset_vis1['links'] = link_list
    print('> Saved: links')

    return dataset_vis1


vis1_data = get_dataset_vis1(tx_db, punk_db, trader_db)
with open('{}/vis1_data.json'.format(VIS_DATA_PATH), 'w') as f:
    json.dump(vis1_data, f)
vis1_data


> Saved: nodes
>> Saved: links between type, gender, and skin_tone
>> Saved: links between skin_tone and attributes
> Saved: links


{'nodes': [{'node': 0, 'name': 'Human'},
  {'node': 1, 'name': 'Zombie'},
  {'node': 2, 'name': 'Ape'},
  {'node': 3, 'name': 'Alien'},
  {'node': 4, 'name': 'Female'},
  {'node': 5, 'name': 'Male'},
  {'node': 6, 'name': '3D Glasses'},
  {'node': 7, 'name': 'Bandana'},
  {'node': 8, 'name': 'Beanie'},
  {'node': 9, 'name': 'Big Beard'},
  {'node': 10, 'name': 'Big Shades'},
  {'node': 11, 'name': 'Black Lipstick'},
  {'node': 12, 'name': 'Blonde Bob'},
  {'node': 13, 'name': 'Blonde Short'},
  {'node': 14, 'name': 'Blue Eye Shadow'},
  {'node': 15, 'name': 'Buck Teeth'},
  {'node': 16, 'name': 'Cap'},
  {'node': 17, 'name': 'Cap Forward'},
  {'node': 18, 'name': 'Chinstrap'},
  {'node': 19, 'name': 'Choker'},
  {'node': 20, 'name': 'Cigarette'},
  {'node': 21, 'name': 'Classic Shades'},
  {'node': 22, 'name': 'Clown Eyes Blue'},
  {'node': 23, 'name': 'Clown Eyes Green'},
  {'node': 24, 'name': 'Clown Hair Green'},
  {'node': 25, 'name': 'Clown Nose'},
  {'node': 26, 'name': 'Cowboy H